Matrices

Una matriz es una colección de números bidimensional. Representaremos
las matrices como listas de listas, teniendo cada lista interior el mismo
tamaño y representando una fila de la matriz. 

Si A es una matriz, entonces
A[i][j] es el elemento de la fila i y la columna j. Por convenio matemático,
utilizaremos con frecuencia letras mayúsculas para representar matrices. Por
ejemplo:

In [ ]:
from typing import List

In [ ]:
# Another type alias
Matrix = List[List[float]]

A = [[1, 2, 3],  # A has 2 rows and 3 columns
     [4, 5, 6]]

B = [[1, 2],     # B has 3 rows and 2 columns
     [3, 4],
     [5, 6]]


Nota: En matemáticas, se suele denominar a la primera fila de la matriz “fila 1”
y a la primera columna “columna 1”. Como estamos representando matrices
con list de Python, que están indexadas al 0, llamaremos a la primera fila de
la matriz “fila 0” y a la primera columna “columna 0”.

Dada esta representación de lista de listas, la matriz A tiene len(A) filas y
len(A[0]) columnas, lo que consideramos su shape:

In [ ]:

from typing import Tuple

def shape(A: Matrix) -> Tuple[int, int]:
    """Returns (# of rows of A, # of columns of A)"""
    num_rows = len(A)
    num_cols = len(A[0]) if A else 0   # number of elements in first row
    return num_rows, num_cols

assert shape([[1, 2, 3], [4, 5, 6]]) == (2, 3)  # 2 rows, 3 columns


Si una matriz tiene n filas y k columnas, nos referiremos a ella como una
matriz n x k. Podemos (y a veces lo haremos) pensar en cada fila de una
matriz n x k como un vector de longitud k, y en cada columna como en un
vector de longitud n:

In [ ]:

def get_row(A: Matrix, i: int) -> Vector:
    """Returns the i-th row of A (as a Vector)"""
    return A[i]             # A[i] is already the ith row

def get_column(A: Matrix, j: int) -> Vector:
    """Returns the j-th column of A (as a Vector)"""
    return [A_i[j]          # jth element of row A_i
            for A_i in A]   # for each row A_i


También querremos poder crear una matriz dada su forma y una función para generar sus elementos. Podemos hacer esto utilizando una comprensión
de lista anidada:

In [ ]:

from typing import Callable

def make_matrix(num_rows: int,
                num_cols: int,
                entry_fn: Callable[[int, int], float]) -> Matrix:
    """
    Returns a num_rows x num_cols matrix
    whose (i,j)-th entry is entry_fn(i, j)
    """
    return [[entry_fn(i, j)             # given i, create a list
             for j in range(num_cols)]  #   [entry_fn(i, 0), ... ]
            for i in range(num_rows)]   # create one list for each i


Dada esta función, se podría crear una matriz identidad 5 x 5 (con unos en
la diagonal y ceros en el resto) de esta forma:

In [ ]:

def identity_matrix(n: int) -> Matrix:
    """Returns the n x n identity matrix"""
    return make_matrix(n, n, lambda i, j: 1 if i == j else 0)

assert identity_matrix(5) == [[1, 0, 0, 0, 0],
                              [0, 1, 0, 0, 0],
                              [0, 0, 1, 0, 0],
                              [0, 0, 0, 1, 0],
                              [0, 0, 0, 0, 1]]


Las matrices serán importantes para nosotros por varias razones.
En primer lugar, podemos utilizar una matriz para representar un conjunto
de datos formado por varios vectores, simplemente considerando cada vector
como una fila de la matriz. Por ejemplo, si tuviéramos las alturas, pesos y
edades de 1.000 personas, podríamos poner estos datos en una matriz 1.000 x
3

In [ ]:

data = [[70, 170, 40],
        [65, 120, 26],
        [77, 250, 19],
        # ....
       ]


En segundo lugar, como veremos más tarde, podemos utilizar una matriz n x k para representar una función lineal que transforme vectores de k
dimensiones en vectores de n dimensiones. Varias de nuestras técnicas y conceptos implicarán tales funciones.

Tercero, las matrices se pueden utilizar para representar relaciones
binarias. En el capítulo 1, representamos los bordes de una red como una
colección de pares (i, j). Una representación alternativa sería crear una
matriz A tal que A[i][j] sea 1 si los nodos i y j están conectados y 0 en otro
caso.

Recordemos que antes teníamos:

In [ ]:

friendships = [(0, 1), (0, 2), (1, 2), (1, 3), (2, 3), (3, 4),
               (4, 5), (5, 6), (5, 7), (6, 8), (7, 8), (8, 9)]


También podemos representar esto como:

In [ ]:

#            user 0  1  2  3  4  5  6  7  8  9
#
friend_matrix = [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0],  # user 0
                 [1, 0, 1, 1, 0, 0, 0, 0, 0, 0],  # user 1
                 [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],  # user 2
                 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0],  # user 3
                 [0, 0, 0, 1, 0, 1, 0, 0, 0, 0],  # user 4
                 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],  # user 5
                 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],  # user 6
                 [0, 0, 0, 0, 0, 1, 0, 0, 1, 0],  # user 7
                 [0, 0, 0, 0, 0, 0, 1, 1, 0, 1],  # user 8
                 [0, 0, 0, 0, 0, 0, 0, 0, 1, 0]]  # user 9


Si hay pocas conexiones, esta representación es mucho menos eficaz, ya
que terminamos teniendo que almacenar muchos ceros. No obstante, con la
representación en matriz se comprueba mucho más rápido si dos nodos están
conectados (basta con hacer una búsqueda en la matriz en lugar de,
posiblemente, inspeccionar cada borde):

In [ ]:

assert friend_matrix[0][2] == 1, "0 and 2 are friends"
assert friend_matrix[0][8] == 0, "0 and 8 are not friends"


De forma similar, para encontrar las conexiones de un nodo, basta con
inspeccionar la columna (o la fila) correspondiente a ese nodo:

In [ ]:

# only need to look at one row
friends_of_five = [i
                   for i, is_friend in enumerate(friend_matrix[5])
                   if is_friend]


Con un gráfico de pequeño tamaño se podría añadir una lista de
conexiones a cada objeto de nodo para acelerar este proceso, pero, con uno
más grande y en constante evolución, hacer esto sería probablemente
demasiado caro y difícil de mantener